In [1]:
library(tidyverse)
library(lubridate)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [3]:
in_path <- "../../lora/data"

nmin_raw <- read.csv(file.path(in_path, "true_data/nmin_raw.csv"), sep = ";", na.strings = c("#VALUE!", "NA", "#DIV/0!"))

crops <- nmin_raw %>% 
    mutate(date = dmy(Date)) %>% 
    select(c(Crop, date, patchID))

sample_data <- nmin_raw %>% 
    mutate(date = dmy(Date),
           depth = recode(Depth,
                         '0-30' = 30,
                         '30-60' = 60,
                         '60-90' = 90,
                         .default = NA_real_)) %>% 
    select(c(patchID, date, depth, kgNmin)) %>% 
    pivot_wider(names_from = depth, values_from = kgNmin, names_prefix = 'nmin_', values_fn = mean) %>% 
    mutate(doy = yday(date)) %>% 
    select(-c(nmin_NA))%>% 
    left_join(crops)%>% 
    rename(patch = patchID, crop = Crop) %>% 
    distinct() %>% 
    filter(!is.na(date))

write.csv(sample_data,
          file = "merge_ready/sample_mr.csv",
          row.names = FALSE)

Joining with `by = join_by(patchID, date)`


In [4]:
sensor_raw <- read.csv(file.path(in_path, "sensor_data_clean_cal/all_sensor_data_clean_cal.csv"))

In [5]:
sensor_data <- sensor_raw %>% 
    mutate(dateTime = as.POSIXct(dateTime, format = "%Y-%m-%dT%H:%M", tz = "UTC"),
           date = as.Date(dateTime)) %>%
    pivot_wider(names_from = c(side, depth), values_from = cal_wc ) %>%
    rowwise() %>% 
    mutate(mc30 = mean(c(left_30, right_30), na.rm = TRUE),
           mc60 = mean(c(left_60, right_60), na.rm = TRUE),
           mc90 = mean(c(left_90, right_90), na.rm = TRUE)) %>%
    select(-c(dateTime,matches("left|right"))) %>% 
    filter(!(is.na(mc30) & is.na(mc60) & is.na(mc90))) %>%
    group_by(date, patch) %>% 
    summarise(sm30 = mean(mc30, na.rm = TRUE),
              sm60 = mean(mc60, na.rm = TRUE),
              sm90 = mean(mc90, na.rm = TRUE))

write.csv(sensor_data,
          file = "merge_ready/sensor_mr.csv",
          row.names = FALSE)

`summarise()` has grouped output by 'date'. You can override using the
`.groups` argument.


In [6]:
#only merges when dates match exactly
sample_sensor_naive <- sample_data %>% 
    left_join(sensor_data, by = c('date', 'patch')) %>%
    arrange(date, patch)

#merge rolled back, last available sensor data used
past <- join_by(patch, closest(x$date >= y$date))
sample_sensor_past <- sample_data %>%
    left_join(sensor_data, past) %>%
    mutate(diff_days = as.numeric(difftime(date.x, date.y, units = 'days'))) %>%
    filter(!is.na(sm30)) %>%
    filter(diff_days < 2)

#merge rolled forwads, next available sensor data used
future <- join_by(patch, closest(x$date <= y$date))
sample_sensor_future <- sample_data %>%
    left_join(sensor_data, future) %>%
    mutate(diff_days = as.numeric(difftime(date.x, date.y, units = 'days'))) %>%
    filter(!is.na(sm30)) %>%
    filter( -2 < diff_days & diff_days < 2)

print(paste("naive has", dim(sample_sensor_naive)[1], "rows"))
print(paste("past has", dim(sample_sensor_past)[1], "rows"))
print(paste("future has", dim(sample_sensor_future)[1], "rows"))


[1] "naive has 261 rows"


[1] "past has 194 rows"
[1] "future has 190 rows"


In [7]:
by <- join_by(patch, date.x)

#merge past and future rolls, take average and remove duplicated nmin, doy, crop info
sample_sensor_full <- sample_sensor_past %>% 
    left_join(sample_sensor_future, by) %>% #join past and future
    rowwise() %>%
    mutate(
        sm_30 = mean(c(sm30.x, sm30.y), na.rm = TRUE),
        sm_60 = mean(c(sm60.x, sm60.y), na.rm = TRUE),#caclulate rowwise means
        sm_90 = mean(c(sm90.x, sm90.y), na.rm = TRUE))

# samples where different dates of sensor values are combined
# for reference
diffs <- sample_sensor_full %>% filter(diff_days.x != diff_days.y)

#only relevent information for final merge
sample_sensor_mr <- sample_sensor_full %>%
    select(-contains(".y") & -c(sm30.x, sm60.x, sm90.x, diff_days.x)) %>%
    rename_with(~ gsub(".x", "", .x, fixed=TRUE)) %>%
    select(-c(nmin_30, nmin_60)) #removes nmin at low depths, comment out to keep that data

write.csv(sample_sensor_mr,
          file = "merge_ready/sample_sensor_mr.csv",
          row.names = FALSE)